In [6]:
from itertools import cycle
from random import choice

class Field:
    # nine spaces in empty field
    free = ' '
    def_cell = list(free * 9)
    
    def __init__(self, data = def_cell):
        self.data = data
        self.curr_step = 'X'
        self.last_step = 'O'
        self.step_dict = {'X': 'O', 'O': 'X'}
        self.step = cycle('OX')
        self.win = False
        self.player_dict = {'user': self.user_move, 
                            'easy': self.easy_move,
                            'medium': self.medium_move,
                            'hard': self.hard_move}
      
    def change_coord(self):
        # check void or full cell in field (for user input)
        cel_num = (self.new_coord[0] - 1) + (3 - self.new_coord[1]) * 3
        if self.data[cel_num] == Field.free:
            # player is changed with move
            self.bare_change(cel_num)
            return True
        print('This cell is occupied! Choose another one!')
        return False
    
    def bare_change(self, cel_num):
        self.data[cel_num], self.last_step = self.curr_step, self.curr_step
        self.curr_step = next(self.step)
        
    def input_coord(self, inp_list):
        # check property of input (2 int digits in range(1,4))
        print(inp_list)
        # shiuld input directly 2 numbers
        if len(inp_list) != 2:
            print('You should enter numbers!')
            return False
        try:
            coord = [int(i) for i in inp_list]
        except ValueError:
            print('You should enter numbers!')
            return False
        
        test = {1, 2, 3}
        # check values
        if not all([x in test for x in coord]):
            print('Coordinates should be from 1 to 3!')
            return False
        
        self.new_coord = coord
        if self.change_coord():
            return True
        return False
        
    def __str__(self):
        # print field with border
        ret_str = '-' * 9 + '\n'
        for i in range(3):
            ret_str += '| ' + ' '.join(self.data[i*3:(i+1)*3]) + ' |' + '\n'
        ret_str += '-' * 9
        return ret_str
    
    def check_win(self, data_lst, step):
        # function checks all rows, columns and diagonals for specified symbol on got list
        r_tup = (0, 1, 2)
        for i in r_tup:
            if all([x == step for x in data_lst[i*3:(i+1)*3]]):
                return True
            if all([x == step for x in [data_lst[i+3*j] for j in r_tup]]):
                return True
        if all([x == step for x in [data_lst[4*i] for i in r_tup]]):
            return True
        if all([x == step for x in [data_lst[2+i*2] for i in r_tup]]):
            return True
        return False
                
    def user_move(self):
        while True:
            if self.input_coord(input('Enter the coordinates:').split()):
                break
                
    def easy_move(self):
        easy_step = choice([i for i in range(9) if self.data[i] == Field.free])
        print('Making move level "easy"')
        self.bare_change(easy_step)
        
    def medium_move(self):
        # work with local data list (it is changed in function)
        med_step = None
        # count curent step
        stp = 9 - self.data.count(Field.free)
        if stp > 2:
            # checking win for every cell, stop if it has found
            for i in [n for n, j in enumerate(self.data) if j == Field.free]:
                med_list = self.data[:]
                med_list[i] = self.curr_step
                if self.check_win(med_list, self.curr_step):
                    med_step = i
                    break
            else:
                # checking lose condition for every cell, stop if it has found
                for i in [n for n, j in enumerate(self.data) if j == Field.free]:
                    med_list = self.data[:]
                    med_list[i] = self.last_step
                    if self.check_win(med_list, self.last_step):
                        med_step = i
                        break
        # random move in other cases        
        if med_step is None:
            med_step = choice([i for i in range(9) if self.data[i] == Field.free])
        print('Making move level "medium"')
        self.bare_change(med_step)

    def min_max(self, field_list, step):
        summ_mm = []
        if self.check_win(field_list, self.curr_step):
            summ_mm.append(10)
            
        elif self.check_win(field_list, self.last_step):
            summ_mm.append(-10)
            
        elif Field.free not in field_list:
            summ_mm.append(0)
            
        else:
            for i in [n for n, j in enumerate(field_list) if j == Field.free]:
                next_list = field_list[:]
                next_list[i] = step
                summ_mm.append(self.min_max(next_list, self.step_dict[step]))
        if step == self.curr_step:
            return max(summ_mm)
        else:
            return min(summ_mm)
        
    def hard_move(self):
        val_dict = dict()
        win = False
        for i in [n for n, j in enumerate(self.data) if j == Field.free]:
            hard_list = self.data[:]
            hard_list[i] = self.curr_step
            val_dict[i] = self.min_max(hard_list, self.last_step)

        hard_step = sorted(val_dict.items(), key = lambda kv: -kv[1])[0][0]
        self.bare_change(hard_step)
    
    def end_game(self):
        if self.check_win(self.data, self.last_step):
            self.win = True
            print(f'{self.last_step} wins')
        return self.win

    def check_input(self, inp_list):
        flag = True        
        in_set = {'start', 'exit'}
        if len(inp_list) != 1 and len(inp_list) != 3:
            flag = False
        elif inp_list[0] not in in_set:
            flag = False
        elif any([x not in self.player_dict.keys() for x in inp_list[1:3]]):
            flag = False
        if not flag:
            print('Bad parameters!')
        return flag

    

while True:
    game = Field(list(' ' * 9))
    command = input('Input command:').split()
    if not game.check_input(command):
        continue
    if command[0] == 'exit':
        break
        
    player1 = game.player_dict[command[1]]
    player2 = game.player_dict[command[2]]

    print(game)
    while Field.free in game.data:
    
        player1()
        print(game)
        if game.end_game():
            break
        if Field.free not in game.data:
            continue
        player2()
        print(game)
        if game.end_game():
            break  
            
    else:
        print('Draw')


Input command:start hard hard
---------
|       |
|       |
|       |
---------
---------
| X     |
|       |
|       |
---------
---------
| X     |
|   O   |
|       |
---------
---------
| X X   |
|   O   |
|       |
---------
---------
| X X O |
|   O   |
|       |
---------
---------
| X X O |
|   O   |
| X     |
---------
---------
| X X O |
| O O   |
| X     |
---------
---------
| X X O |
| O O X |
| X     |
---------
---------
| X X O |
| O O X |
| X O   |
---------
---------
| X X O |
| O O X |
| X O X |
---------
Draw
Input command:start hard hard
---------
|       |
|       |
|       |
---------
---------
| X     |
|       |
|       |
---------
---------
| X     |
|   O   |
|       |
---------
---------
| X X   |
|   O   |
|       |
---------
---------
| X X O |
|   O   |
|       |
---------
---------
| X X O |
|   O   |
| X     |
---------
---------
| X X O |
| O O   |
| X     |
---------
---------
| X X O |
| O O X |
| X     |
---------
---------
| X X O |
| O O X |
| X O

In [7]:
def dep(val):
    i = 0
    while val < 700000:
        i += 1
        val *= 1.071
    return i
print(dep(int(input())))

650000
2
